# Base inventory env

> Base environment with some basic funcitons

In [1]:
#| default_exp envs.inventory.inventory_utils

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
from abc import ABC, abstractmethod
from typing import Union, Tuple

from ddopnew.envs.base import BaseEnvironment
from ddopnew.utils import Parameter, MDPInfo
from ddopnew.dataloaders.base import BaseDataLoader
from ddopnew.loss_functions import pinball_loss

import gymnasium as gym

import numpy as np
import time

In [4]:
#| export
class OrderPipeline():
    """
    Class to handle the order pipeline in the inventory environments. It is used to keep track of the orders
    that are placed. It can account for fixed and variable lead times.
    
    """

    def __init__(self, 

        num_units: int,  # number of units (SKUs)
        lead_time_mean: list[object] | None = None,  # default is empty list
        
        max_lead_time: list[object] | None = None,  # maximum lead time in case of stochastic lead times

        ) -> None:

        if max_lead_time is None:
            self.max_lead_time = lead_time_mean
        else:
            self.max_lead_time = max_lead_time

        self.pipeline = np.zeros((np.max(max_lead_time), num_units))

    def step(self, 
        orders: np.ndarray,
        ) -> np.ndarray:
        
        """ Add orders to the pipeline and return the orders that are arriving """


In [7]:
testpipeline = OrderPipeline(lead_time_mean=[1, 2, 3], max_lead_time=[3, 4, 5])

testpipeline.max_lead_time


AttributeError: 'OrderPipeline' object has no attribute 'max_lead_time'

In [6]:
#| hide
import nbdev; nbdev.nbdev_export()